In [1]:
from epsilon_transformers.process.processes import Mess3
from epsilon_transformers.process.MixedStateTree import MixedStateTree, MixedStateTreeNode
from epsilon_transformers.process.Process import _compute_emission_probabilities, _compute_next_distribution, Process
import numpy as np

In [2]:
x = 0.15
a = 0.54

In [3]:
import itertools
import numpy as np

def generate_combinations():
    combinations = []
    for length in range(10,11):  # Lengths from 2 to 10
        for combo in itertools.product([0, 1, 2], repeat=length):
            combinations.append(np.array(combo))
    
    return combinations

# Generate the combinations
combinations_list = generate_combinations()


In [4]:
import torch
import time

def multiply_beliefs(batch1: torch.Tensor, batch2: torch.Tensor):
    accumulated_tm = torch.einsum('bij,cjk->bcik', batch1, batch2)
    num_symbols, num_seqs, n, _ = accumulated_tm.shape
    accumulated_tm = accumulated_tm.reshape(num_symbols * num_seqs, n, n)
    return accumulated_tm

def compute_all_beliefs(process: Process, seq_len: int, device=torch.device("cpu")):
    tm = torch.tensor(process.transition_matrix, dtype=torch.float32, device=device)
    accumulated_tm = torch.eye(len(tm), dtype=torch.float32, device=device).unsqueeze(0)

    for i in range(seq_len):
        accumulated_tm = multiply_beliefs(tm, accumulated_tm)

    return accumulated_tm

def compute_all_beliefs_with_history(process: Process, seq_len: int, device=torch.device("cpu")):
    tm = torch.tensor(process.transition_matrix, dtype=torch.float32, device=device)
    accumulated_tm = torch.eye(len(tm), dtype=torch.float32, device=device).unsqueeze(0)

    layers = []
    for i in range(seq_len):
        accumulated_tm = multiply_beliefs(tm, accumulated_tm)
        layers.append(accumulated_tm.repeat_interleave((tm.shape[-1])**(seq_len - i - 1), dim=0))

    return layers

st = time.time()
prior = torch.tensor([1/3, 1/3, 1/3], device="cpu")
raw_beliefs = compute_all_beliefs_with_history(Mess3(x, a), 10, device=torch.device("cpu"))

states = prior @ torch.stack(raw_beliefs).permute(1,0,2,3)
states = states / torch.sum(states, dim=2).unsqueeze(2)

print(time.time() - st)

0.032945871353149414


In [5]:
from src.generate_paths_and_beliefs import generate_mess3_beliefs

beliefs_source = generate_mess3_beliefs(x, a, sort_pairs=True)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 2],
        ...,
        [2, 2, 2,  ..., 2, 2, 0],
        [2, 2, 2,  ..., 2, 2, 1],
        [2, 2, 2,  ..., 2, 2, 2]], dtype=torch.int32)


In [6]:
beliefs_source[1]

tensor([[[0.5400, 0.2300, 0.2300],
         [0.6549, 0.1725, 0.1725],
         [0.7098, 0.1451, 0.1451],
         ...,
         [0.7514, 0.1243, 0.1243],
         [0.7518, 0.1241, 0.1241],
         [0.7519, 0.1241, 0.1241]],

        [[0.5400, 0.2300, 0.2300],
         [0.6549, 0.1725, 0.1725],
         [0.7098, 0.1451, 0.1451],
         ...,
         [0.7514, 0.1243, 0.1243],
         [0.7518, 0.1241, 0.1241],
         [0.4354, 0.3960, 0.1687]],

        [[0.5400, 0.2300, 0.2300],
         [0.6549, 0.1725, 0.1725],
         [0.7098, 0.1451, 0.1451],
         ...,
         [0.7514, 0.1243, 0.1243],
         [0.7518, 0.1241, 0.1241],
         [0.4354, 0.1687, 0.3960]],

        ...,

        [[0.2300, 0.2300, 0.5400],
         [0.1725, 0.1725, 0.6549],
         [0.1451, 0.1451, 0.7098],
         ...,
         [0.1243, 0.1243, 0.7514],
         [0.1241, 0.1241, 0.7518],
         [0.3960, 0.1687, 0.4354]],

        [[0.2300, 0.2300, 0.5400],
         [0.1725, 0.1725, 0.6549],
         [0.

In [7]:
(states - beliefs_source[1])[55463]

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def ternary_to_cartesian(ternary_coords):
    # Convert ternary coordinates to Cartesian coordinates for plotting
    cartesian_coords = []
    for (a, b, c) in ternary_coords:
        x = 0.5 * (2 * b + c) / (a + b + c)
        y = (np.sqrt(3) / 2) * c / (a + b + c)
        cartesian_coords.append((x, y))
    return np.array(cartesian_coords)

def plot_ternary(ternary_coords):
    cartesian_coords = ternary_to_cartesian(ternary_coords)
    x = cartesian_coords[:, 0]
    y = cartesian_coords[:, 1]
    
    fig, ax = plt.subplots()
    ax.scatter(x, y,s =0.01)

    # Draw the ternary plot boundaries
    ax.plot([0, 0.5], [0, np.sqrt(3) / 2], 'k-')
    ax.plot([0.5, 1], [np.sqrt(3) / 2, 0], 'k-')
    ax.plot([1, 0], [0, 0], 'k-')

    ax.set_xlim(0, 1)
    ax.set_ylim(0, np.sqrt(3) / 2)
    ax.set_aspect('equal')
    ax.set_axis_off()

    plt.show()

plot_ternary(states)

ValueError: too many values to unpack (expected 3)